# Importing and Setting Up GDrive

In [ ]:
import pandas as pd
import numpy as np
import jax.numpy as jnp
import jax

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Change directory here!
%cd /content/drive/MyDrive/dataset/DSA4212

/content/drive/MyDrive/dataset/DSA4212


# Reading Data

In [ ]:
train_all = pd.read_csv('assignment_2_ratings_train.csv')
print(train_all.shape)
train_all.head()

(4436068, 3)


,user_id,anime_id,rating
0,20170,10794,6
1,24592,21995,5
2,18358,7054,10
3,59267,488,7
4,69313,30544,4


In [ ]:
anime = pd.read_csv('assignment_2_anime.csv')
print(anime.shape)
anime.head()

(12294, 7)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
train, valid = train_test_split(train_all, test_size=0.2)
valid = valid.reset_index(drop=True)

# Average by genres

For each user, we compute the average rating they gave to an anime of each genre. For each user and anime we are predicting, we consider the genres of the anime and the known average ratings for the user for these genres, and predict the rating as the average of the known genres. If the user has no known ratings for any animes of the genres of the anime we are predicting, we then predict the global average.



In [ ]:
global_avg = np.mean(train['rating']) #global average

train_new = train.copy()
train_new = train_new.merge(anime[["anime_id", "genre"]], on = "anime_id")
train_new = train_new.dropna()
train_new["genre"] = train_new["genre"].apply(lambda x: x.split(", "))
train_new = train_new.explode("genre").groupby(["user_id", "genre"])[["rating"]].mean()
train_new = train_new.reset_index()
train_new = train_new.rename(columns={'rating': 'mean_genre'})

valid_new = valid.copy()
valid_new = valid_new.merge(anime[["anime_id", "genre"]], how = "left", on = "anime_id")
valid_new["genre"] = valid_new["genre"].apply(lambda x: pd.NA if pd.isna(x) else x.split(", "))
valid_new = valid_new.explode("genre")

In [ ]:
valid_new = valid_new.merge(train_new, on = ["user_id", "genre"], how = "left")
pred = valid_new.groupby(["user_id", "anime_id", "rating"], sort = False).mean("mean_genre").reset_index()
pred[pred["mean_genre"].isna()] = global_avg

In [ ]:
print("Validation MSE: ", np.mean((pred["rating"]-pred["mean_genre"]) **2))

Validation MSE:  1.7469341163348429
